In [2]:
#======================================================功能性函数=======================================================
def create_data(day,node_set,LMP_data):#取出每天的电价
    dict_LMP={}
    for i in node_set:
        dict_LMP[i]=LMP_data[i][day]
    return dict_LMP

def Distance(s1, s2):
    return sqrt(sum((s1-s2)**2))

def dist(node1,node2,para,dict_LMP):#计算两个点之间的时空距离
    return Distance(dict_LMP[node1],dict_LMP[node2])*(duration[node_set.index(node1)][node_set.index(node2)]**0.5)

def sel_center(node_list,dict_LMP,para):#选取中心点
    list2=[]
    for i in node_list:
        list2.append((i,sum([dist(i,j,para,dict_LMP) for j in node_list])))
    list2=sorted(list2,key=lambda x: x[1])
    return list2[0][0]

def k_means(node_set,k,para,dict_LMP):
    core_list=[]
    new_core=random.sample(node_set,k)
    t=1
    while set(new_core)!=set(core_list) and t<100:
        t+=1
        core_list=new_core
        new_core=[]
        clf_list=[]
        for i in range(k):
            clf_list.append([core_list[i]])      
        for i in node_set:
            list1=[]
            for j in range(len(core_list)):
                list1.append((j,dist(i,core_list[j],para,dict_LMP)))
            list1=sorted(list1,key=lambda x: x[1])
            clf_list[list1[0][0]].append(i)
        clf_list=[list(set(i)) for i in clf_list]
        for i in clf_list:
            new_core.append(sel_center(i,dict_LMP,para))
    return core_list,clf_list
#========================================================聚类函数================================================================
'''
def cluster(node_set,LMP_data,para,day,k):
    dict_LMP=create_data(day,node_set,LMP_data)
    core_list,clf_list=k_means(node_set,k,para,dict_LMP)
    out_list=[]
    for i in range(k):
        df_empty = pd.DataFrame(columns=clf_list[i])
        list1=[]
        list2=[]
        for nods in clf_list[i]:
            df_empty[nods]=dict_LMP[nods]
        df_mean=np.array(df_empty.mean()).mean()
        for j in range(24):
            array1=np.array(df_empty.iloc[j,:])
            array1=array1-df_mean
            index_max=list(abs(array1)).index(max(abs(array1)))
            list1.append(df_empty.columns[index_max])
            list2.append(array1[index_max]+df_mean)
        out_list.append(max(list1,key=list1.count))
    return out_list
'''
#第一种构造虚拟电价的方法，只考虑当天同一个聚类情况下的点。这个地方的days可以是一个含有很多日期的列表。
def add_max_difference_column(dataframe):
    mean_values = dataframe.mean(axis=1)
    differences = dataframe.sub(mean_values, axis=0).abs()
    max_difference_column = differences.idxmax(axis=1)
    dataframe['Max_Difference'] = dataframe.apply(lambda row: row[max_difference_column[row.name]], axis=1)
    dataframe['Name'] = max_difference_column
    return dataframe

def cluster_1(node_set,LMP_data,para,days,k):
    out_dict={}
    for day in days:
        dict_LMP=create_data(day,node_set,LMP_data)
        core_list,clf_list=k_means(node_set,k,para,dict_LMP)
        sumofdist=0
        for i in range(k):
            sumofdist+=sum([dist(core_list[i],t,1,dict_LMP) for t in clf_list[i]])
        tmp_dict={}
        for i in range(k):
            df_empty = pd.DataFrame(columns=clf_list[i])
            list1=[]
            list2=[]
            for nods in clf_list[i]:
                df_empty[nods]=dict_LMP[nods]
            df_empty=add_max_difference_column(df_empty)
            fake_price = copy.deepcopy(df_empty[[core_list[i],'Max_Difference','Name']])
            tmp_dict[core_list[i]] = fake_price
        out_dict[day]=tmp_dict
    
    return out_dict,sumofdist

def cluster_2(node_set,LMP_data,para,days,k):#在cluster1上的改进，聚类能够更加紧密。
    out_dict={}
    for day in days:
        dict_LMP=create_data(day,node_set,LMP_data)
        sort_list=[]
        for i in range(10):
            core_list,clf_list=k_means(node_set,k,para,dict_LMP)
            sumofdist=0
            for i in range(k):
                sumofdist+=sum([dist(core_list[i],t,1,dict_LMP) for t in clf_list[i]])
            sort_list.append((core_list,clf_list,sumofdist))
        sorted_list = sorted(sort_list, key=lambda x: x[-1])
        core_list,clf_list,sumofdist = sorted_list[0]
        tmp_dict={}
        for i in range(k):
            df_empty = pd.DataFrame(columns=clf_list[i])
            list1=[]
            list2=[]
            for nods in clf_list[i]:
                df_empty[nods]=dict_LMP[nods]
            df_empty=add_max_difference_column(df_empty)
            fake_price = copy.deepcopy(df_empty[[core_list[i],'Max_Difference','Name']])
            tmp_dict[core_list[i]] = fake_price
        out_dict[day]=tmp_dict
    
    return out_dict,sumofdist
'''
def cluster_test(node_set,LMP_data,para,days,k):#只做聚类不计算
    out_dict={}
    for day in days:
        dict_LMP=create_data(day,node_set,LMP_data)
        sort_list=[]
        for i in range(10):
            core_list,clf_list=k_means(node_set,k,para,dict_LMP)
            sumofdist=0
            for i in range(k):
                sumofdist+=sum([dist(core_list[i],t,1,dict_LMP) for t in clf_list[i]])
            sort_list.append((core_list,clf_list,sumofdist))
        sorted_list = sorted(sort_list, key=lambda x: x[-1])
        core_list,clf_list,sumofdist = sorted_list[0]
        return sumofdist
'''
#第三种聚类方法，部分地实现自适应，在给定聚类中心数并获得out_dict后，根据out_dict的特性决定是不是需要将其中的某个聚类分裂成两个
#修改alter函数或者生成out_dict的函数，不能使用聚类中心的地理坐标，而是应该使用那些被访问最多的点。
from collections import Counter
def sel_node(out_dict):#这个函数将保证out_dict中的每一个聚类所取的地理位置都更加合适
    new_dict = dict()
    for days in out_dict:
        original_day_dict = out_dict[days]
        new_day_dict=dict()
        for nodes in original_day_dict:
            temp_list = list(original_day_dict[nodes]['Name'])
            counter = Counter(temp_list)
            geo_node = counter.most_common(1)[0][0]
            new_day_dict[geo_node] = original_day_dict[nodes]
        new_dict[days] = new_day_dict
    
    return new_dict
def cluster_3(node_set,LMP_data,para,days,k):#在cluster2上的改进
    out_dict={}
    for day in days:
        dict_LMP=create_data(day,node_set,LMP_data)
        sort_list=[]
        for i in range(10):
            core_list,clf_list=k_means(node_set,k,para,dict_LMP)
            sumofdist=0
            for i in range(k):
                sumofdist+=sum([dist(core_list[i],t,1,dict_LMP) for t in clf_list[i]])
            sort_list.append((core_list,clf_list,sumofdist))
        sorted_list = sorted(sort_list, key=lambda x: x[-1])
        core_list,clf_list,sumofdist = sorted_list[0]
        tmp_dict={}
        for i in range(k):
            df_empty = pd.DataFrame(columns=clf_list[i])
            list1=[]
            list2=[]
            for nods in clf_list[i]:
                df_empty[nods]=dict_LMP[nods]
            df_empty=add_max_difference_column(df_empty)
            fake_price = copy.deepcopy(df_empty[[core_list[i],'Max_Difference','Name']])
            tmp_dict[core_list[i]] = fake_price
        
        
        out_dict[day]=tmp_dict
        out_dict = sel_node(out_dict)
    return out_dict,sumofdist

def cluster_4(node_set,LMP_data,para,days,k):#在cluster3上的改进,实现自适应的聚类,此处k为下限
    out_dict={}
    for day in days:
        dict_LMP=create_data(day,node_set,LMP_data)
        Dataframe_LMP = pd.DataFrame(dict_LMP)
        sort_list=[]
        for i in range(10):
            core_list,clf_list=k_means(node_set,k,para,dict_LMP)
            sumofdist=0
            for i in range(k):
                sumofdist+=sum([dist(core_list[i],t,1,dict_LMP) for t in clf_list[i]])
            sort_list.append((core_list,clf_list,sumofdist))
        sorted_list = sorted(sort_list, key=lambda x: x[-1])
        core_list,clf_list,sumofdist = sorted_list[0]
        def gen_tmp_dict(clf_list,core_list,dict_LMP):
            k = len(clf_list)
            tmp_dict={}
            for i in range(k):
                df_empty = pd.DataFrame(columns=clf_list[i])
                list1=[]
                list2=[]
                for nods in clf_list[i]:
                    df_empty[nods]=dict_LMP[nods]
                df_empty=add_max_difference_column(df_empty)
                fake_price = copy.deepcopy(df_empty[[core_list[i],'Max_Difference','Name']])
                tmp_dict[core_list[i]] = fake_price
            return tmp_dict
        tmp_dict = gen_tmp_dict(clf_list,core_list,dict_LMP)
        #此处已经形成了一个tmp_dict，还有core_list,clf_list,接下来的问题是如何自适应地不断更新clf_list
        signal = 1
        while signal == 1:
            keys_list = list(tmp_dict.keys())
            signal_list = [0]*len(keys_list)#用来表示每一个聚类是否被修改，最后只有当全部为0时才把signal置0
            for i in range(len(signal_list)):
                cluster_data = tmp_dict[keys_list[i]]
                #先挑出最多的那个点代表肯定要
                list1 = list(cluster_data['Name'])
                counter = Counter(list1)
                geo_node = counter.most_common(1)[0][0]
                for j in range(23):
                    if list1[j]!=geo_node and (cluster_data['Max_Difference'][j] == sorted(list(Dataframe_LMP.iloc[j]))[0] or cluster_data['Max_Difference'][j] == sorted(list(Dataframe_LMP.iloc[j]))[1] or cluster_data['Max_Difference'][j] == sorted(list(Dataframe_LMP.iloc[j]))[-1] or cluster_data['Max_Difference'][j] == sorted(list(Dataframe_LMP.iloc[j]))[-2]):
                        signal_list[i]=1
                        #print(i)
                        #print(clf_list[i])
                        clf_list[i].remove(list1[j])
                        clf_list.append([list1[j]])
                        break
            if sum(signal_list)==0:
                signal = 0
            core_list = [t[0] for t in clf_list]
            tmp_dict = gen_tmp_dict(clf_list,core_list,dict_LMP)
                                                                   
        out_dict[day]=tmp_dict
        out_dict = sel_node(out_dict)
    return clf_list,out_dict,sumofdist

def cluster_4_1(node_set,LMP_data,para,days,k):#在cluster3上的改进,实现自适应的聚类,此处k为下限
    out_dict={}
    for day in days:
        dict_LMP=create_data(day,node_set,LMP_data)
        Dataframe_LMP = pd.DataFrame(dict_LMP)
        sort_list=[]
        for i in range(10):
            core_list,clf_list=k_means(node_set,k,para,dict_LMP)
            sumofdist=0
            for i in range(k):
                sumofdist+=sum([dist(core_list[i],t,1,dict_LMP) for t in clf_list[i]])
            sort_list.append((core_list,clf_list,sumofdist))
        sorted_list = sorted(sort_list, key=lambda x: x[-1])
        core_list,clf_list,sumofdist = sorted_list[0]
        def gen_tmp_dict(clf_list,core_list,dict_LMP):
            k = len(clf_list)
            tmp_dict={}
            for i in range(k):
                df_empty = pd.DataFrame(columns=clf_list[i])
                list1=[]
                list2=[]
                for nods in clf_list[i]:
                    df_empty[nods]=dict_LMP[nods]
                df_empty=add_max_difference_column(df_empty)
                fake_price = copy.deepcopy(df_empty[[core_list[i],'Max_Difference','Name']])
                tmp_dict[core_list[i]] = fake_price
            return tmp_dict
        tmp_dict = gen_tmp_dict(clf_list,core_list,dict_LMP)
        #此处已经形成了一个tmp_dict，还有core_list,clf_list,接下来的问题是如何自适应地不断更新clf_list
        signal = 1
        while signal == 1:
            keys_list = list(tmp_dict.keys())
            signal_list = [0]*len(keys_list)#用来表示每一个聚类是否被修改，最后只有当全部为0时才把signal置0
            for i in range(len(signal_list)):
                cluster_data = tmp_dict[keys_list[i]]
                #先挑出最多的那个点代表肯定要
                list1 = list(cluster_data['Name'])
                counter = Counter(list1)
                geo_node = counter.most_common(1)[0][0]
                for j in range(23):
                    if list1[j]!=geo_node and (cluster_data['Max_Difference'][j] == sorted(list(Dataframe_LMP.iloc[j]))[0] or cluster_data['Max_Difference'][j] == sorted(list(Dataframe_LMP.iloc[j]))[-1]):
                        signal_list[i]=1
                        #print(i)
                        #print(clf_list[i])
                        clf_list[i].remove(list1[j])
                        clf_list.append([list1[j]])
                        break
            if sum(signal_list)==0:
                signal = 0
            core_list = [t[0] for t in clf_list]
            tmp_dict = gen_tmp_dict(clf_list,core_list,dict_LMP)
                                                                   
        out_dict[day]=tmp_dict
        out_dict = sel_node(out_dict)
    return clf_list,out_dict,sumofdist

#如果out_dict中的点为当前时间最大或者最小的电价，则为其单开一列
from itertools import chain
def cluster_5(node_set,LMP_data,para,days,k,signal):#希望返回一个这样的字典{day1：【clf_list1】,...}
    clflist_dict={}
    for day in days:
        key_set = set()
        for i in range(10):
            if signal==1:
                clf_list,out_dict, sumofdist = cluster_4_1(node_set, LMP_data, 1, [day], k)
            elif signal==0:
                clf_list,out_dict, sumofdist = cluster_4(node_set, LMP_data, 1, [day], k)       
            keys = out_dict[day].keys()
            key_set.update(keys)
        clflist_dict[day] = list(key_set)
    return clflist_dict

#===========================================================聚类后的调整与利润计算============================================================
import copy 
def alter(out_dict,LMP_data):#out_dict中存放的字典实例{81{node1：}}
    #为了防止基础的LMP_data被改变
    copy_LMP = copy.deepcopy(LMP_data)
    days=list(out_dict.keys())
    for i in days:
        tmp_dict=out_dict[i]
        for node in tmp_dict.keys():
            copy_LMP[node][i]=np.array(tmp_dict[node]['Max_Difference'])
    return copy_LMP

def cal_pro(df):
    pro=0
    list1=list(df.columns)
    for i in range(len(list1)//2):
        pro = pro+sum(np.array(df[list1[i]])*np.array(df[list1[i+len(list1)//2]]))
    return pro

def chg_outdict(out_dict,node_set,duration):
    #第一步应该是计算相关参数，但是因为多天非常难操作先考虑单天，故将一大堆东西省略
    list1=[]
    list2=[]
    for day in list(out_dict.keys()):
        day_dict=out_dict[day]
        out_set=list(out_dict[day].keys())
        [duration_1,distance] = travel_data_cal(out_set,node_data)
        dataframe=get_Dataframe(duration_1,out_set,day,day_dict)
        list1.append('第'+str(day)+'天在删除不可能路径前的利润为'+str(cal_pro(dataframe)))
        list2.append(cal_pro(dataframe))
        dataframe=chg(duration,node_set,duration_1,out_set,day_dict,dataframe)
        list1.append('第'+str(day)+'天删除不可能路径后的利润为'+str(cal_pro(dataframe)))
        list2.append(cal_pro(dataframe))
    return list1,list2
    
        
def get_Dataframe(duration_1,out_set,day,day_dict):
    day_set=[day]
    g_dict={}

    g_dict[day]=day_dict
    [model2,sta_profit,sta_usage,sta_profit_reserve,elapsed_time2,G2,PESS_cha,PESS_dis,SESS_energy,PESS_energy,RES_cap,record_PESS_cha_dis,PESS_REP_travel_grid,PESS_travel_grid] = data_driven_model_solve(u,mcu,c_tr,num_veh,day_set,scale,out_set,duration_1,AS_type,alter(g_dict,LMP_data),price_data_SR,price_data_NR,save_dir,power_cap,energy_cap,effi,n,SOH,solver_option,timelimit,save_file_flag,c_index,print_log,rep_log,price_differ,RT_flag)
    new_value = np.zeros(shape = (num_veh,len(PESS_travel_grid[0][0]),96))
    for ii in range(1,num_veh+1):
        for i in range(0,len(PESS_travel_grid[ii-1][0])):
            for j in range(0,96):
                new_value[ii-1][i][j] = round(model2.energy_veh_discharged['v'+str(ii),PESS_travel_grid[ii-1][0][i],j].value,4) -round(model2.energy_veh_charged['v'+str(ii),PESS_travel_grid[ii-1][0][i],j].value,4)
        dataframe1 = pd.DataFrame({'Energy at Node ' + str(PESS_travel_grid[ii-1][0][node_id]): new_value[ii-1][node_id] for node_id in range(0,len(new_value[ii-1]))})
        dataframe2 = pd.DataFrame({'LMP at Node ' + str(PESS_travel_grid[ii-1][0][node_id]): model2.price[PESS_travel_grid[ii-1][0][node_id]] for node_id in range(0,len(new_value[ii-1]))})
        dataframe =pd.concat([dataframe1, dataframe2], axis=1, join='inner')
        return dataframe

def chg(duration,node_set,duration_1,out_set,day_dict,dataframe):
    #第一步，把day_dict的格式规范化为96行的
    for node in day_dict.keys():
        tmpdf = day_dict[node]
        tmpdf = pd.DataFrame(np.repeat(tmpdf.values,4,axis=0), columns=tmpdf.columns)
        day_dict[node] = tmpdf
    #第二步，整理出一条小车的路径
    dataframe_half = dataframe.iloc[:, :len(list(dataframe.columns))//2]
    lst=[]
    for i in list(dataframe_half.columns):
        num = int(i[-1])-1
        node = list(day_dict.keys())[num]
        rout_lst = list(day_dict[node]['Name'])
        sig_lst = list(dataframe_half[i])        
        result_list = [rout_lst[j] if sig_lst[j]!=0 else 0 for j in range(len(rout_lst))]
        lst.append(result_list)
    route_list = [next((item for item in items if item != 0), 0) for items in zip(*lst)]
    #第三步，修改route_list
    def interval(node_1,node_2,duration,node_set):
        index_1 = node_set.index(node_1)
        index_2 = node_set.index(node_2)
        return duration[index_1][index_2]        
    last_non_zero = None
    for i in range(len(route_list)):
        if route_list[i] != 0:  # If element is not 0
            if last_non_zero is None:
                last_non_zero = i
            elif last_non_zero is not None:  # If there was a non zero element before it
                if interval(route_list[i],route_list[last_non_zero],duration,node_set)<=15*(i-last_non_zero-1):
                    last_non_zero = i
                elif interval(route_list[i],route_list[last_non_zero],duration,node_set)>15*(i-last_non_zero-1):
                    route_list[i]=0
    #第四步修改dataframe
    signal_list = [1 if i != 0 else 0 for i in route_list]
    for i in list(dataframe_half.columns):
        dataframe[i] = np.array(dataframe[i])*np.array(signal_list)
    return dataframe

In [3]:
#!/usr/bin/env python
# coding: utf-8

# In[1]:


from __future__ import division
import pandas as pd
import requests, zipfile, io
import pytz, datetime
from io import BytesIO
from datetime import timedelta
import numpy as np
import time
import json
import os, sys
import pickle
from pprint import pprint
from pyomo.environ import *
import random
import pandas as pd
import matplotlib.pyplot as plt
import cloudpickle
from IPython.display import clear_output
from tqdm import tnrange, tqdm_notebook

start_time = datetime.datetime.now().strftime('%Y%m%d_%H%M%S')
print('Current time:',start_time)
%set_env GUROBI_HOME=/usr/local/gurobi/gurobi951/linux64
    
# from sta.bat_sta import *
from sta.traffic_data_fetch import *
from sta.load_data import *
# from sta.load_node_set import *
from sta.node_data_processing import *
from sta.node_distance_cal import *

import networkx as nx

print('Functions loaded.')


# In[2]:


cwd = os.getcwd()
data_path = cwd +'/Data/'
results_dir = cwd+'/results/'
save_dir = cwd+'/results/'

par = pd.read_csv(data_path + "parameter_setting.csv")
# node_data = pd.read_csv(data_path + "node.csv")
pkl_file = open(data_path+'node_data_clean.pkl', 'rb')
node_data = pickle.load(pkl_file)
pkl_file.close()

[location_data,node_id_set,duration_data,distance_data,LMP_data,LMP_data_RTM,price_data_SR,price_data_NR] = load_data(data_path)


# In[4]:


timelimit = 100
solver_option = 0 ## 0:Gurobi, 1:CPLEX, 2:GAMS
radius = 7

total_node_num = len(node_data)

sample_node_set = [36]


for sample_number in sample_node_set:
    # for c in idx:
    c_index = int(sample_number)
    print('Task ID:', c_index)
    center_node_name = node_data['node_name'][c_index]
    print('Center Node:', center_node_name)
    print('Radius:', radius)
    print('Timelimit:', timelimit)
   
#     random.seed(2020)
#     mylist = range(total_node_num)
    # sample_node_set = random.sample(mylist,200)
    

    ## Inputs
    u_min = par['u_min'][0]
    u_max = par['u_max'][0]
    u_scale = par['u_scale'][0]

    u_sensi = par['u_sensi'][0]

    AS_type = par['AS_type'][0]
    RT_flag = par['RT_flag'][0]
    day_num = par['day_num'][0]
    full_day_set = np.arange(day_num)

    scale = par['scale'][0]
    energy_cap = par['energy_cap'][0]
    power_cap = par['power_cap'][0]
    # power_cap = 0
    # energy_cap = 0
    effi = par['effi'][0]
    discount_rate = par['discount_rate'][0]
    mcu = par['mcu'][0]
    calendar_degradation = par['calendar_degradation'][0]
    usage_limit = par['cycle_life'][0]*energy_cap*2*day_num/365
    c_tr = par['c_tr'][0]*scale
    cost_rep = 10*scale #换电成本

    save_file_flag = 1

    set_region_range = 0
    
    if set_region_range == 0:
        center_node = center_node_name

        node_set = load_node_set_circle(center_node,radius,node_data)
    elif set_region_range == 1:

        node_set = ['KETTLEMN_6_N001','HURON_6_N001']

    for cent_node in range(len(node_set)):
        if node_set[cent_node] == 'KEARNEY_7_N001':    #n26
            Grid2 = cent_node

#     center_node = center_node_name
#     node_set = load_node_set_circle(center_node,radius,node_data)
    
    print('Node Set loaded.')
    node_num = len(node_set)
    print('Number of nodes:', node_num)
    print('SESS Node: ',node_set[Grid2])


    [duration,distance] = travel_data_cal(node_set,node_data)
    # [duration,distance] = travel_data_fetch(node_set,node_data)
    # print('Duration matrix: ',duration)





def data_driven_model_solve(u,mcu,c_tr,num_veh,day_set,scale,node_set,duration,AS_type,
                LMP_data,price_data_SR,price_data_NR,
                save_dir,power_cap,energy_cap,effi,year_num,SOH,
                solver_option,timelimit,save_file_flag,c_index,print_log,rep_log,price_differ):



    node_num = len(node_set)
    center_node_num = [0] #中心点的数量，即SESS的数量


    sta_profit = []
    sta_travel = []
    sta_usage = []
    sta_reserve_rev = []
    elapsed_time2 = []

    initial_location = [0] * node_num
    initial_location[0] = 1


    for day in day_set:

        start_time = time.time()

        p_da = []

        for i in range(len(node_set)):
            p_da.append(LMP_data[node_set[i]][day])
       
        
        if AS_type == 'SR':
            p_reserve = price_data_SR[day]
        else:
            p_reserve = price_data_NR[day] 

        hour_num = len(p_da[0])


        model = ConcreteModel()
        model.max_power = power_cap * scale #SESS最大功率，乘以scale，后面在计算soc就不用再乘以了scale
        model.max_energy = energy_cap #SESS最大容量
        model.max_interval_energy = power_cap * scale #PESS最大功率
        model.max_vehicle_energy = energy_cap  #PESS最大容量
        model.eta=effi

        horizon = model.horizon = int(hour_num/scale)
#         horizon = model.horizon = int(2)
        model.node_num = node_num


        p_da_rescale = np.zeros(shape=(len(node_set),horizon))
        p_reserve_rescale = np.zeros(horizon)
        for n in range(node_num):
            for t in range(horizon):
                p_da_rescale[n][t] = p_da[n][floor(t*scale)]  
                p_reserve_rescale[t] = p_reserve[floor(t*scale)]

        model.h = scale
        model.Hrzn = RangeSet(0,model.horizon-1)
        Grid = model.Grid = ['n'+str(x) for x in range(1,model.node_num+1)]
        #Grid_center = model.Grid[Grid2]
        


        keys = Grid
        values = p_da_rescale
        model.price = dict(zip(keys, values)) 
        model.price_reserve = p_reserve_rescale



        

        if type(duration) == dict:
            travel_time = np.ceil(duration[day]/(60*scale))
        else:
            travel_time = np.ceil(duration/(60*scale))
        travel_time = travel_time.astype(int)

        model.travel_time = dict(zip(keys,[dict(zip(keys, values)) for values in travel_time]))


        model.c = mcu #mcu
        model.c_tr = c_tr #单位运输成本
        model.usage_limit = model.max_interval_energy / scale * 2400
        #model.calendar_d = model.max_energy*0.01/365
        model.calendar_d = 0  ##日历老化，如果周期为一天，则可以设为0。如果是长时间尺度的求解，一般可以设置为50

    
 
        G = nx.DiGraph()
        G.add_node('source', grid=None, time=0)
        G.add_node('sink', grid=None, time=model.horizon) # the source and sink node are added for convenience
        # node_set = set(['source','sink'])
        for t in range(0, model.horizon):
            for grid_name in Grid:
                G.add_node('%s_%d' % (grid_name, t), grid=grid_name, time=t)

        # node_set.update('%s_%d' % (grid_name, t) for grid_name in Grid  for t in range(0, model.horizon))

        # suppose we start from grid 1, change Grid[0] if starting from a different grid
        G.add_edge('source', '%s_%d' % (Grid[0], 0), arc_type='entering', cost= 0)   #‘entering’表示起始点（进入点）
        for t1 in range(0, model.horizon):  
            for grid1_name in Grid:   #弧的起点
                cur_grid = '%s_%d' % (grid1_name,t1)
                for grid2_name in Grid:  #弧的终点
                    if grid2_name==grid1_name:
                        t2 = t1 + 1
                    else:
                        t2 = t1 + model.travel_time[grid1_name][grid2_name] +1
                        
                        if t2 < model.horizon:   #如果电价差不够大的话，就不把这条弧加进来（因为路径规划的求解是遍历每一条可能的弧，弧的数量越多，求解越慢，这一步的目的是加速模型求解）
                            if abs(model.price[grid1_name][t1]-model.price[grid2_name][t2])<price_differ:  #这一步虽然能加速算法的求解，但是可能会把存在更优解的弧给删掉了，提高了速度，但损失了精度
                                continue
                    if t2 >= model.horizon:
                        continue
                    
                    
                    target_grid = '%s_%d' % (grid2_name,t2) 
                    G.add_edge(cur_grid, target_grid, arc_type='inter',  #‘inter’表示中间点
                        cost=model.travel_time[grid1_name][grid2_name]*model.c_tr*model.h) #计算当前时刻每一个点到下一个点所需的时间和由此产生的运输成本

        for grid1_name in Grid:  
            G.add_edge('%s_%d' % (grid1_name, model.horizon-1), 'sink', arc_type='leaving', cost= 0)  #‘leaving’表示终止点（离开点）

#从这里开始，数据驱动模型跟原来的模型Time-space-network中不一致  

        for t in range(0, model.horizon):
            for grid_name in Grid:
                node_name = '%s_%d' % (grid_name, t)
                if len(G.out_edges(node_name)) == 0 and len(G.in_edges(node_name)) == 0:
                    G.remove_node(node_name)

        # nx.draw_networkx(G, with_label = True) 
        
#从这里开始，数据驱动模型跟原来的模型Time-space-network中不一致 

        
        veh_lmt = 3 #每个点最多允许3辆车同时充放电
        veh_name = ['v'+str(x) for x in range(1,num_veh+1)]

        model.profit = Var(domain=Reals)
        model.revenue = Var(domain=Reals)
        model.reserve_revenue = Var(domain=Reals)
        model.cost_sto = Var(domain=Reals)
        model.cost_tr = Var(domain=Reals)

########## 考虑SESS充放电和PESS换电
        if rep_log == 1:
            model.x = Var(veh_name,G.edges,domain=Binary)
            # model.y = Var(veh_name,model.Grid,model.Hrzn,domain=Binary)#换电标志
            model.z = Var(veh_name,model.Grid,model.Hrzn,domain=Binary)#PESS充放电标志
            #以下两式被替换
            model.energy = Var(model.Grid,model.Hrzn, domain=NonNegativeReals,bounds=(0,model.max_energy)) #SESS的容量,对应约束条件（11），这里做了松弛，因为考虑了换电，所以变量的上界不能单纯只是储能电站的容量
            # model.energy = Var(model.Grid,model.Hrzn, domain=NonNegativeReals) #SESS的容量,对应约束条件（11），这里做了松弛，因为考虑了换电，所以变量的上界不能单纯只是储能电站的容量
            model.vehicle_energy = Var(veh_name, model.Hrzn, domain=NonNegativeReals,bounds=(0,model.max_vehicle_energy)) #PESS的容量,对应约束条件（10）
            model.energy_discharged = Var(model.Grid,model.Hrzn, domain=NonNegativeReals,bounds=(0,model.max_interval_energy)) #SESS的放电功率，对应约束条件（8）
            model.energy_charged = Var(model.Grid,model.Hrzn, domain=NonNegativeReals,bounds=(0,model.max_interval_energy))  #SESS的充电功率，对应约束条件（8）
            model.energy_veh_discharged = Var(veh_name,model.Grid,model.Hrzn, domain=NonNegativeReals,bounds=(0,model.max_interval_energy)) #PESS的放电功率
            model.energy_veh_charged = Var(veh_name,model.Grid,model.Hrzn, domain=NonNegativeReals,bounds=(0,model.max_interval_energy))#PESS的充电功率
           
            model.energy_rep_discharged = Var(veh_name,model.Grid,model.Hrzn, domain=NonNegativeReals,bounds=(0,model.max_vehicle_energy)) #换电的放电功率，可能是有问题的，本应该是离散变量，这里我们连续化了
            model.energy_rep_charged = Var(veh_name,model.Grid,model.Hrzn, domain=NonNegativeReals,bounds=(0,model.max_vehicle_energy))#换电的充电功率，可能是有问题的，本应该是离散变量，这里我们连续化了

            model.reserve_cap = Var(model.Grid,model.Hrzn, domain=NonNegativeReals,bounds=(0,model.max_interval_energy)) #上限是设为model.max_interval_energy 还是model.max_energy，或者不设限？？
            model.usage = Var(domain=NonNegativeReals,bounds=(0,model.usage_limit))


            def obj_value(model):
                return model.profit

            def obj_function(model):   
                return model.profit == model.revenue-model.cost_sto-model.cost_tr + model.reserve_revenue
            model.OF = Constraint(rule=obj_function)


            def revenue_cal(model):      
                return model.revenue == -sum(sum(sum((model.energy_rep_discharged[v,g,t] + model.energy_rep_charged[v,g,t])*cost_rep for g in model.Grid) for t in model.Hrzn) for v in veh_name) + sum(sum(sum((model.energy_veh_discharged[v,g,t]-model.energy_veh_charged[v,g,t])*model.price[g][t] for g in model.Grid) for t in model.Hrzn) for v in veh_name) + sum(sum((model.energy_discharged[g,t]-model.energy_charged[g,t])*model.price[g][t] for g in model.Grid) for t in model.Hrzn)
            model.rev = Constraint(rule=revenue_cal)

            def reserve_revenue_cal(model): #需不需要加上PESS的辅助服务和SESS的辅助服务
                return model.reserve_revenue == sum(sum( model.reserve_cap[g,t] * model.price_reserve[t] for g in model.Grid) for t in model.Hrzn)
            model.reserve_rev = Constraint(rule=reserve_revenue_cal)

            def cost_storage_cal(model):  
                return model.cost_sto == model.usage*model.c
            model.cost_s = Constraint(rule=cost_storage_cal)

            def cost_transportation_cal(model):  
                return model.cost_tr == sum(sum(model.x[v,arc]*G.edges[arc]['cost'] for arc in G.edges) for v in veh_name)
            model.cost_t = Constraint(rule=cost_transportation_cal)

            def usage(model):  
                return model.usage == sum(sum(sum((model.energy_veh_discharged[v,g,t]+model.energy_veh_charged[v,g,t]) for g in model.Grid) for t in model.Hrzn) for v in veh_name) + sum(sum((model.energy_discharged[g,t] + model.energy_charged[g,t]) for g in model.Grid) for t in model.Hrzn) + model.calendar_d
            model.power_usage = Constraint(rule=usage)

            def balance(model,v,g):  #对应约束条件（1）
                if g=='source':
                    return sum(model.x[v,arc] for arc in G.out_edges(g))  == 1
                elif g=='sink':
                    return sum(model.x[v,arc] for arc in G.in_edges(g))  == 1
                else:
                    return sum(model.x[v,arc] for arc in G.out_edges(g)) - sum(model.x[v,arc] for arc in G.in_edges(g))  == 0
            model.flow_balance = Constraint(veh_name, G.nodes, rule=balance)


            def soc_change(model,v,t): # 对应约束条件（3），还要加上换电
                if t==0:
                    return model.vehicle_energy[v,t] == model.vehicle_energy[v,model.horizon-1]-sum(model.energy_veh_discharged[v,g,t] for g in model.Grid)/model.eta + sum(model.energy_veh_charged[v,g,t] for g in model.Grid)*model.eta + sum(model.energy_rep_charged[v,g,t] for g in model.Grid) - sum(model.energy_rep_discharged[v,g,t] for g in model.Grid)
                else:
                    return model.vehicle_energy[v,t] == model.vehicle_energy[v,t-1]-sum(model.energy_veh_discharged[v,g,t] for g in model.Grid)/model.eta + sum(model.energy_veh_charged[v,g,t] for g in model.Grid)*model.eta + sum(model.energy_rep_charged[v,g,t] for g in model.Grid) - sum(model.energy_rep_discharged[v,g,t] for g in model.Grid)
            model.soc = Constraint(veh_name, model.Hrzn, rule=soc_change)

            #定义其他节点的SOC等于0
            def soc_change3(model,g,t):  #除了SESS，其他点的SOC恒等于0
                if g!=Grid_center:  
                    return model.energy[g,t] == 0
                else:
                    if t==0:  
                        return model.energy[g,t] == model.energy[g,model.horizon-1]- model.energy_discharged[g,t]/model.eta + model.energy_charged[g,t]*model.eta + sum(model.energy_rep_discharged[v,g,t] for v in veh_name) - sum(model.energy_rep_charged[v,g,t] for v in veh_name) 
                    else:
                        return model.energy[g,t] == model.energy[g,t-1]- model.energy_discharged[g,t]/model.eta + model.energy_charged[g,t]*model.eta + sum(model.energy_rep_discharged[v,g,t] for v in veh_name) - sum(model.energy_rep_charged[v,g,t] for v in veh_name)
            model.soc3 = Constraint(model.Grid,model.Hrzn, rule=soc_change3)

            #只有SESS才能充放电
            def energy_dis_location(model,g,t):  
                if g!=Grid_center:  
                    return model.energy_discharged[g,t] == 0
                else:
                    return model.energy_discharged[g,t] <= model.max_interval_energy
            model.energy_dis_con = Constraint(model.Grid,model.Hrzn, rule=energy_dis_location)

            def energy_cha_location(model,g,t):  
                if g!=Grid_center:  
                    return model.energy_charged[g,t] == 0
                else:
                    return model.energy_charged[g,t] <= model.max_interval_energy
            model.energy_cha_con = Constraint(model.Grid,model.Hrzn, rule=energy_cha_location)

            def veh_dis_location(model,v,g,t):  #对应约束条件（6）
                if g==Grid_center:
                    return model.energy_veh_discharged[v,g,t] == 0
                else:
                    return model.energy_veh_discharged[v,g,t] <= model.z[v,g,t] * model.max_interval_energy
            model.veh_dis_con = Constraint(veh_name, model.Grid,model.Hrzn, rule=veh_dis_location)  

            def veh_cha_location(model,v,g,t):  #对应约束条件（6）
                if g==Grid_center:
                    return model.energy_veh_charged[v,g,t] == 0
                else:
                    return model.energy_veh_charged[v,g,t] <= model.z[v,g,t] * model.max_interval_energy    
            model.veh_cha_con = Constraint(veh_name, model.Grid,model.Hrzn, rule=veh_cha_location)  

            #定义除了SESS，其他点的换电变量等于0
            def rep_dis_location(model,v,g,t):  #除了SESS，其他点的SESS恒等于0
                if g!=Grid_center:  
                    return model.energy_rep_discharged[v,g,t] == 0
                else:
                    return model.energy_rep_discharged[v,g,t] <= model.z[v,g,t] * model.max_energy
            model.rep_dis_con = Constraint(veh_name, model.Grid,model.Hrzn, rule=rep_dis_location)

            def rep_cha_location(model,v,g,t):  #除了SESS，其他点的SESS恒等于0
                if g!=Grid_center:  
                    return model.energy_rep_charged[v,g,t] == 0
                else:
                    return model.energy_rep_charged[v,g,t] <= model.z[v,g,t] * model.max_energy
            model.rep_cha_con = Constraint(veh_name, model.Grid,model.Hrzn, rule=rep_cha_location)

            def reserve_location(model,v,g,t): #这个约束要考虑一下，可能要加上SESS的辅助服务
                return model.reserve_cap[g,t] <= model.z[v,g,t] * model.max_interval_energy
            model.p_reserve = Constraint(veh_name,model.Grid,model.Hrzn, rule=reserve_location)

            def storage_up(model,g,t):   #对于储能电站来说，是减去放电加上充电？
                return model.reserve_cap[g,t] + sum((model.energy_veh_discharged[v,g,t] - model.energy_veh_charged[v,g,t]) for v in veh_name) <= model.max_interval_energy               
            model.stor_up = Constraint(model.Grid,model.Hrzn, rule=storage_up)

            def storage_down(model,g,t):  #对于储能电站来说，是减去放电加上充电？
                return model.reserve_cap[g,t] + sum((model.energy_veh_discharged[v,g,t] - model.energy_veh_charged[v,g,t]) for v in veh_name) >= -model.max_interval_energy    
            model.stor_down = Constraint(model.Grid, model.Hrzn, rule=storage_down)      
            
            def veh_storage_location(model,v,g,t):  #对应约束条件（5）
                g_arc = g+'_'+str(t)
                return model.z[v,g,t] <= sum(model.x[v,arc] for arc in G.in_edges(g_arc)) 
            model.veh_sto_location = Constraint(veh_name,model.Grid, model.Hrzn, rule=veh_storage_location)


    #从这里开始，数据驱动模型跟原来的模型Time-space-network中不一致          
            # def veh_limit(model,g,t):   #对应约束条件（4）
            #     return sum(model.z[v,g,t] + model.y[v,g,t] for v in veh_name) <= veh_lmt
            # model.veh_limit = Constraint(model.Grid, model.Hrzn, rule=veh_limit)           

            def veh_limit(model,g,t):   #对应约束条件（4）
                return sum(model.z[v,g,t] for v in veh_name) <= veh_lmt
            model.veh_limit = Constraint(model.Grid, model.Hrzn, rule=veh_limit)

            def soc_veh_initial(model,v):  #PESS的soc_initial
                return model.vehicle_energy[v,0] == 0
            model.soc1 = Constraint(veh_name, rule=soc_veh_initial)
            #可以考虑把这个松弛掉
            # def soc_initial(model,g):  #SESS的soc_initial，设定等于充满电？
            #     return model.energy[g,0] == 0
            # model.soc4 = Constraint(model.Grid, rule=soc_initial)

    #从这里开始，数据驱动模型跟原来的模型Time-space-network中不一致  


########## 不考虑SESS充放电和PESS换电
        if rep_log == 0:
            model.x = Var(veh_name,G.edges,domain=Binary)
            model.z = Var(veh_name,model.Grid,model.Hrzn,domain=Binary) #相当于PESS代码中的 m.w
            model.vehicle_energy = Var(veh_name, model.Hrzn, domain=NonNegativeReals,bounds=(0,model.max_vehicle_energy)) #PESS的SOC,对应约束条件（10）
            model.energy_veh_discharged = Var(veh_name,model.Grid,model.Hrzn, domain=NonNegativeReals,bounds=(0,model.max_interval_energy)) #PESS的放电功率
            model.energy_veh_charged = Var(veh_name,model.Grid,model.Hrzn, domain=NonNegativeReals,bounds=(0,model.max_interval_energy))#PESS的充电功率

            model.reserve_cap = Var(model.Grid,model.Hrzn, domain=NonNegativeReals,bounds=(0,model.max_interval_energy)) #上限是设为model.max_interval_energy 
            model.usage = Var(domain=NonNegativeReals,bounds=(0,model.usage_limit))
            
            def obj_value(model):
                return model.profit

            def obj_function(model):   
                return model.profit == model.revenue-model.cost_sto-model.cost_tr + model.reserve_revenue
            model.OF = Constraint(rule=obj_function)

            def revenue_cal(model):      
                return model.revenue == sum(sum(sum((model.energy_veh_discharged[v,g,t]-model.energy_veh_charged[v,g,t])*model.price[g][t] for g in model.Grid) for t in model.Hrzn) for v in veh_name)
            model.rev = Constraint(rule=revenue_cal)

            def reserve_revenue_cal(model): 
                return model.reserve_revenue == sum(sum( model.reserve_cap[g,t] * model.price_reserve[t] for g in model.Grid) for t in model.Hrzn)
            model.reserve_rev = Constraint(rule=reserve_revenue_cal)

            def cost_storage_cal(model):  
                return model.cost_sto == model.usage*model.c
            model.cost_s = Constraint(rule=cost_storage_cal)

            def cost_transportation_cal(model): 
                return model.cost_tr == sum(sum(model.x[v,arc]*G.edges[arc]['cost'] for arc in G.edges) for v in veh_name)
            model.cost_t = Constraint(rule=cost_transportation_cal)

            def usage(model):  
                return model.usage == sum(sum(sum((model.energy_veh_discharged[v,g,t]+model.energy_veh_charged[v,g,t]) for g in model.Grid) for t in model.Hrzn) for v in veh_name) + model.calendar_d
            model.power_usage = Constraint(rule=usage)

            def balance(model,v,g):  #对应约束条件（1）
                if g=='source':
                    return sum(model.x[v,arc] for arc in G.out_edges(g))  == 1
                elif g=='sink':
                    return sum(model.x[v,arc] for arc in G.in_edges(g))  == 1
                else:
                    return sum(model.x[v,arc] for arc in G.out_edges(g)) - sum(model.x[v,arc] for arc in G.in_edges(g))  == 0
            model.flow_balance = Constraint(veh_name, G.nodes, rule=balance)

            def soc_change(model,v,t): # 对应约束条件（3），还要加上换电
                if t==0:
                    return model.vehicle_energy[v,t] == model.vehicle_energy[v,model.horizon-1]-sum(model.energy_veh_discharged[v,g,t] for g in model.Grid)/model.eta + sum(model.energy_veh_charged[v,g,t] for g in model.Grid)*model.eta 
                else:
                    return model.vehicle_energy[v,t] == model.vehicle_energy[v,t-1]-sum(model.energy_veh_discharged[v,g,t] for g in model.Grid)/model.eta + sum(model.energy_veh_charged[v,g,t] for g in model.Grid)*model.eta 
            model.soc = Constraint(veh_name, model.Hrzn, rule=soc_change)

            def veh_dis_location(model,v,g,t):  #对应约束条件（6）
                return model.energy_veh_discharged[v,g,t] <= model.z[v,g,t] * model.max_interval_energy
            model.p_dis = Constraint(veh_name, model.Grid,model.Hrzn, rule=veh_dis_location)  

            def veh_cha_location(model,v,g,t):  #对应约束条件（6）
                return model.energy_veh_charged[v,g,t] <= model.z[v,g,t] * model.max_interval_energy    
            model.p_cha = Constraint(veh_name, model.Grid,model.Hrzn, rule=veh_cha_location)              

            def reserve_location(model,g,t):
                return model.reserve_cap[g,t] <= sum(model.z[v,g,t] for v in veh_name) * model.max_interval_energy
            model.p_reserve = Constraint(model.Grid,model.Hrzn, rule=reserve_location)

            def storage_up(model,g,t):   #转成PESS时候，要把右端项改为model.max_power
                return model.reserve_cap[g,t] + sum((model.energy_veh_discharged[v,g,t] - model.energy_veh_charged[v,g,t]) for v in veh_name) <= model.max_interval_energy   
            model.stor_up = Constraint(model.Grid,model.Hrzn, rule=storage_up)

            def storage_down(model,g,t):  #转成PESS时候，要把右端项改为model.max_power
                return model.reserve_cap[g,t] + sum((model.energy_veh_discharged[v,g,t] - model.energy_veh_charged[v,g,t]) for v in veh_name) >= -model.max_interval_energy   
            model.stor_down = Constraint(model.Grid, model.Hrzn, rule=storage_down)        
            
            def storage_location(model,v,g,t):  #对应约束条件（5）
                g_arc = g+'_'+str(t)
                return model.z[v,g,t] <= sum(model.x[v,arc] for arc in G.in_edges(g_arc)) 
            model.sto_location = Constraint(veh_name,model.Grid, model.Hrzn, rule=storage_location)

    #从这里开始，数据驱动模型跟原来的模型Time-space-network中不一致          
            def veh_limit(model,g,t):   #对应约束条件（4）
                return sum(model.z[v,g,t] for v in veh_name) <= veh_lmt
            model.veh_limit = Constraint(model.Grid, model.Hrzn, rule=veh_limit)     

#             def soc_veh_initial(model,v):  #PESS的soc_initial
#                     return model.vehicle_energy[v,0] == 0
#             model.soc1 = Constraint(veh_name, rule=soc_veh_initial)


    #从这里开始，数据驱动模型跟原来的模型Time-space-network中不一致  


        model.OBJ = Objective(rule=obj_value, sense=maximize)

        success_flag = 0
        print_error = 1
        while success_flag == 0:                        
            success_flag = 1                        
            try:
                if solver_option == 2:
                    io_options = dict()
                    opt = SolverFactory('gams')
                    io_options['solver'] = 'cplex'
                    results = opt.solve(model,tee=print_log,io_options=io_options) 
                elif solver_option == 1:

                    opt = SolverFactory('cplex',executable = "/home/gridsan/gnhe/opt/ibm/ILOG/CPLEX_Studio129/cplex/bin/x86-64_linux/cplex")
                    opt.options['timelimit'] = timelimit
                    results = opt.solve(model,tee=print_log)
                else:
                    opt = SolverFactory('gurobi',executable="/usr/local/gurobi/gurobi951/linux64/bin/gurobi.sh")
                    opt.options['timelimit'] = timelimit
#             opt.options['timelimit'] = 20
                    results = opt.solve(model,tee=print_log)
                                
            except Exception as vehicle_energy:
                success_flag = 0
                if print_error == 1:
                    print('Error:',vehicle_energy)
                    print('RetrinG...')
                time.sleep( 5 )


        real_profit = model.profit.value+model.cost_sto.value
        sta_profit.append(model.profit.value+model.cost_sto.value)
        # sta_travel.append(daily_travel)
        sta_usage.append(model.usage.value)
        sta_reserve_rev.append(model.reserve_revenue.value)    

        elapsed_time = time.time() - start_time  
        elapsed_time2.append(elapsed_time)


        print('Task ID:', str(c_index), round(mcu,3), 'Year:',year_num, 'Day:',day, 'Profit:',round(real_profit), 'Elapsed_time',elapsed_time)
    
    if len(day_set) != 1:
        for i in range(len(elapsed_time2)-1):
            elapsed_time2[i+1] = elapsed_time2[i+1] + elapsed_time2[i]
    
    if rep_log == 1:
        return(model,sta_profit,sta_usage,sta_reserve_rev,elapsed_time2,G)
    
    if rep_log == 0:
        return(model,sta_profit,sta_usage,sta_reserve_rev,elapsed_time2,G)



u = 0 
num_veh = 1 #车的数量
day_set = np.arange(365)
# day_set = np.arange(7)
day_set = [0,72,81]
n = 1
SOH = 1
mcu = 0.01 #这里的mcu是Joule文章里的c^{DEG}_t，由MBU除以折扣因子得到（这里的mcu=0.01可能是由MBU=50计算而来(未验证)，MBU=50接近于MBU的最优取值）
rep_log = 0 #是否考虑换电的标志，等于0则为纯PESS。这里的换电表示储能电站（SESS）的充放电的PESS的换电行为
print_log = 0 # print_log = 1 为打印求解过程
price_differ = 0 #电价差，用于加速求解过程，值越大，求解速度越快，但精度可能越差
timelimit = 10000

Current time: 20250712_115341
env: GUROBI_HOME=/usr/local/gurobi/gurobi951/linux64
Functions loaded.
Data loaded.
Task ID: 36
Center Node: MURRAY_6_N005
Radius: 7
Timelimit: 100
Node Set loaded.
Number of nodes: 16
SESS Node:  KEARNEY_7_N001
Travel data calcualted.


In [4]:
results_df = pd.DataFrame(columns=['days', 'sta_profit', 'elapsed_time2', 'out_set'])

clflist_dict = cluster_5(node_set, LMP_data, 1, [i for i in range(0, 365)], 4, 1)
for days in list(clflist_dict.keys()):
    out_set = clflist_dict[days]
    [duration_1, distance] = travel_data_cal(out_set, node_data)
    [model2, sta_profit, sta_usage, sta_profit_reserve, elapsed_time2, G2] = data_driven_model_solve(
        u, mcu, c_tr, num_veh, [days], scale, out_set, duration_1, AS_type, 
        LMP_data, price_data_SR, price_data_NR, save_dir, power_cap, energy_cap, 
        effi, n, SOH, solver_option, timelimit, save_file_flag, c_index, print_log, 
        rep_log, price_differ
    )
    
    # 创建新行DataFrame并使用concat来添加到results_df中
    new_row = pd.DataFrame({
        'days': [days],
        'sta_profit': [sta_profit],
        'elapsed_time2': [elapsed_time2],
        'out_set': [str(out_set)]  # 将列表转化为字符串存储
    })
    results_df = pd.concat([results_df, new_row], ignore_index=True)
    
    
# 将DataFrame保存到Excel文件
    results_df.to_excel('results_4.xlsx', index=False, engine='openpyxl')

Travel data calcualted.
Task ID: 36 0.01 Year: 1 Day: 0 Profit: 114 Elapsed_time 4.284499168395996
Travel data calcualted.
Task ID: 36 0.01 Year: 1 Day: 1 Profit: 133 Elapsed_time 33.81704783439636
Travel data calcualted.
Task ID: 36 0.01 Year: 1 Day: 2 Profit: 308 Elapsed_time 97.1173083782196
Travel data calcualted.
Task ID: 36 0.01 Year: 1 Day: 3 Profit: 803 Elapsed_time 291.27183198928833
Travel data calcualted.
Task ID: 36 0.01 Year: 1 Day: 4 Profit: 358 Elapsed_time 563.230893611908
Travel data calcualted.
Task ID: 36 0.01 Year: 1 Day: 5 Profit: 105 Elapsed_time 128.30159282684326
Travel data calcualted.
Task ID: 36 0.01 Year: 1 Day: 6 Profit: 72 Elapsed_time 3.2849018573760986
Travel data calcualted.
Task ID: 36 0.01 Year: 1 Day: 7 Profit: 565 Elapsed_time 193.91327118873596
Travel data calcualted.
Task ID: 36 0.01 Year: 1 Day: 8 Profit: 718 Elapsed_time 49.295939683914185
Travel data calcualted.
Task ID: 36 0.01 Year: 1 Day: 9 Profit: 790 Elapsed_time 21.907346963882446
Travel 

Travel data calcualted.
Task ID: 36 0.01 Year: 1 Day: 81 Profit: 1256 Elapsed_time 446.18964886665344
Travel data calcualted.
Task ID: 36 0.01 Year: 1 Day: 82 Profit: 231 Elapsed_time 13.561985492706299
Travel data calcualted.
Task ID: 36 0.01 Year: 1 Day: 83 Profit: 188 Elapsed_time 60.69110131263733
Travel data calcualted.
Task ID: 36 0.01 Year: 1 Day: 84 Profit: 253 Elapsed_time 29.255812168121338
Travel data calcualted.
Task ID: 36 0.01 Year: 1 Day: 85 Profit: 290 Elapsed_time 55.568347454071045
Travel data calcualted.
Task ID: 36 0.01 Year: 1 Day: 86 Profit: 271 Elapsed_time 23.74387764930725
Travel data calcualted.
Task ID: 36 0.01 Year: 1 Day: 87 Profit: 334 Elapsed_time 34.26545524597168
Travel data calcualted.
Task ID: 36 0.01 Year: 1 Day: 88 Profit: 3292 Elapsed_time 34.194624185562134
Travel data calcualted.
Task ID: 36 0.01 Year: 1 Day: 89 Profit: 129 Elapsed_time 23.463473796844482
Travel data calcualted.
Task ID: 36 0.01 Year: 1 Day: 90 Profit: 140 Elapsed_time 16.1087322

Travel data calcualted.
Task ID: 36 0.01 Year: 1 Day: 160 Profit: 155 Elapsed_time 11.545982837677002
Travel data calcualted.
Task ID: 36 0.01 Year: 1 Day: 161 Profit: 491 Elapsed_time 116.20715308189392
Travel data calcualted.
Task ID: 36 0.01 Year: 1 Day: 162 Profit: 7086 Elapsed_time 26.12801504135132
Travel data calcualted.
Task ID: 36 0.01 Year: 1 Day: 163 Profit: 908 Elapsed_time 83.92432451248169
Travel data calcualted.
Task ID: 36 0.01 Year: 1 Day: 164 Profit: 359 Elapsed_time 57.93296003341675
Travel data calcualted.
Task ID: 36 0.01 Year: 1 Day: 165 Profit: 540 Elapsed_time 14.947827339172363
Travel data calcualted.
Task ID: 36 0.01 Year: 1 Day: 166 Profit: 218 Elapsed_time 15.135519027709961
Travel data calcualted.
Task ID: 36 0.01 Year: 1 Day: 167 Profit: 158 Elapsed_time 10.314156293869019
Travel data calcualted.
Task ID: 36 0.01 Year: 1 Day: 168 Profit: 900 Elapsed_time 75.57785248756409
Travel data calcualted.
Task ID: 36 0.01 Year: 1 Day: 169 Profit: 2138 Elapsed_time 2

Travel data calcualted.
Task ID: 36 0.01 Year: 1 Day: 241 Profit: 684 Elapsed_time 33.96735906600952
Travel data calcualted.
Task ID: 36 0.01 Year: 1 Day: 242 Profit: 472 Elapsed_time 50.61696147918701
Travel data calcualted.
Task ID: 36 0.01 Year: 1 Day: 243 Profit: 160 Elapsed_time 11.768241882324219
Travel data calcualted.
Task ID: 36 0.01 Year: 1 Day: 244 Profit: 154 Elapsed_time 8.649877548217773
Travel data calcualted.
Task ID: 36 0.01 Year: 1 Day: 245 Profit: 175 Elapsed_time 7.312888145446777
Travel data calcualted.
Task ID: 36 0.01 Year: 1 Day: 246 Profit: 344 Elapsed_time 25.6823308467865
Travel data calcualted.
Task ID: 36 0.01 Year: 1 Day: 247 Profit: 350 Elapsed_time 254.70062351226807
Travel data calcualted.
Task ID: 36 0.01 Year: 1 Day: 248 Profit: 407 Elapsed_time 156.42051339149475
Travel data calcualted.
Task ID: 36 0.01 Year: 1 Day: 249 Profit: 345 Elapsed_time 418.17829608917236
Travel data calcualted.
Task ID: 36 0.01 Year: 1 Day: 250 Profit: 286 Elapsed_time 16.03

Task ID: 36 0.01 Year: 1 Day: 322 Profit: 349 Elapsed_time 16.39191961288452
Travel data calcualted.
Task ID: 36 0.01 Year: 1 Day: 323 Profit: 335 Elapsed_time 9.523799419403076
Travel data calcualted.
Task ID: 36 0.01 Year: 1 Day: 324 Profit: 198 Elapsed_time 29.115975379943848
Travel data calcualted.
Task ID: 36 0.01 Year: 1 Day: 325 Profit: 121 Elapsed_time 9.156553506851196
Travel data calcualted.
Task ID: 36 0.01 Year: 1 Day: 326 Profit: 188 Elapsed_time 8.622585535049438
Travel data calcualted.
Task ID: 36 0.01 Year: 1 Day: 327 Profit: 285 Elapsed_time 10.577873945236206
Travel data calcualted.
Task ID: 36 0.01 Year: 1 Day: 328 Profit: 139 Elapsed_time 8.160620212554932
Travel data calcualted.
Task ID: 36 0.01 Year: 1 Day: 329 Profit: 298 Elapsed_time 14.941021203994751
Travel data calcualted.
Task ID: 36 0.01 Year: 1 Day: 330 Profit: 229 Elapsed_time 20.60164475440979
Travel data calcualted.
Task ID: 36 0.01 Year: 1 Day: 331 Profit: 175 Elapsed_time 7.272716760635376
Travel data

In [ ]:
results_df = pd.DataFrame(columns=['days', 'sta_profit', 'elapsed_time2', 'out_set'])

clflist_dict = cluster_5(node_set, LMP_data, 1, [i for i in range(0, 365)], 4, 1)
for days in list(clflist_dict.keys()):
    out_set = clflist_dict[days]
    [duration_1, distance] = travel_data_cal(out_set, node_data)
    [model2, sta_profit, sta_usage, sta_profit_reserve, elapsed_time2, G2] = data_driven_model_solve(
        u, mcu, c_tr, num_veh, [days], scale, out_set, duration_1, AS_type, 
        LMP_data, price_data_SR, price_data_NR, save_dir, power_cap, energy_cap, 
        effi, n, SOH, solver_option, timelimit, save_file_flag, c_index, print_log, 
        rep_log, price_differ
    )
    
    # 创建新行DataFrame并使用concat来添加到results_df中
    new_row = pd.DataFrame({
        'days': [days],
        'sta_profit': [sta_profit],
        'elapsed_time2': [elapsed_time2],
        'out_set': [str(out_set)]  # 将列表转化为字符串存储
    })
    results_df = pd.concat([results_df, new_row], ignore_index=True)
    
    
# 将DataFrame保存到Excel文件
    results_df.to_excel('results_4_16nodes.xlsx', index=False, engine='openpyxl')

Travel data calcualted.
Task ID: 36 0.01 Year: 1 Day: 0 Profit: 113 Elapsed_time 3.2207536697387695
Travel data calcualted.
Task ID: 36 0.01 Year: 1 Day: 1 Profit: 113 Elapsed_time 8.156650304794312
Travel data calcualted.
Task ID: 36 0.01 Year: 1 Day: 2 Profit: 166 Elapsed_time 4.79353141784668
Travel data calcualted.
Task ID: 36 0.01 Year: 1 Day: 3 Profit: 227 Elapsed_time 18.81339454650879
Travel data calcualted.
Task ID: 36 0.01 Year: 1 Day: 4 Profit: 108 Elapsed_time 5.565033912658691
Travel data calcualted.
Task ID: 36 0.01 Year: 1 Day: 5 Profit: 63 Elapsed_time 2.9181013107299805
Travel data calcualted.
Task ID: 36 0.01 Year: 1 Day: 6 Profit: 72 Elapsed_time 1.757441759109497
Travel data calcualted.
Task ID: 36 0.01 Year: 1 Day: 7 Profit: 275 Elapsed_time 14.72854471206665
Travel data calcualted.
Task ID: 36 0.01 Year: 1 Day: 8 Profit: 131 Elapsed_time 6.042249441146851
Travel data calcualted.
Task ID: 36 0.01 Year: 1 Day: 9 Profit: 368 Elapsed_time 12.386274099349976
Travel dat

Travel data calcualted.
Task ID: 36 0.01 Year: 1 Day: 82 Profit: 223 Elapsed_time 13.808778047561646
Travel data calcualted.
Task ID: 36 0.01 Year: 1 Day: 83 Profit: 160 Elapsed_time 9.041005373001099
Travel data calcualted.
Task ID: 36 0.01 Year: 1 Day: 84 Profit: 233 Elapsed_time 7.428027868270874
Travel data calcualted.
Task ID: 36 0.01 Year: 1 Day: 85 Profit: 224 Elapsed_time 5.608827114105225
Travel data calcualted.
Task ID: 36 0.01 Year: 1 Day: 86 Profit: 201 Elapsed_time 21.29225468635559
Travel data calcualted.
Task ID: 36 0.01 Year: 1 Day: 87 Profit: 167 Elapsed_time 13.1563081741333
Travel data calcualted.
Task ID: 36 0.01 Year: 1 Day: 88 Profit: 3208 Elapsed_time 11.748593807220459
Travel data calcualted.
Task ID: 36 0.01 Year: 1 Day: 89 Profit: 129 Elapsed_time 12.918749570846558
Travel data calcualted.
Task ID: 36 0.01 Year: 1 Day: 90 Profit: 140 Elapsed_time 15.29973578453064
Travel data calcualted.
Task ID: 36 0.01 Year: 1 Day: 91 Profit: 187 Elapsed_time 8.3684058189392

In [4]:
df_result4 = pd.read_excel('results_4.xlsx')

In [10]:
df_result4

,days,sta_profit,elapsed_time2,out_set
0,0,[114.10979568884868],[4.284499168395996],"['KEARNEY_7_N001', 'CLARMNT_1_N001', 'CARLTNHS..."
1,1,[132.71867336707072],[33.81704783439636],"['KEARNEY_7_N001', 'CLARMNT_1_N001', 'ELCAJNGT..."
2,2,[308.4794238280262],[97.1173083782196],"['KEARNEY_7_N001', 'STREAMVW_6_N001', 'CLARMNT..."
3,3,[803.2769083032236],[291.27183198928833],"['KEARNEY_7_N001', 'MIRAMAR_6_N004', 'CLARMNT_..."
4,4,[358.1405476602232],[563.230893611908],"['KEARNEY_7_N001', 'MIRAMAR_6_N004', 'CLARMNT_..."
...,...,...,...,...
360,360,[124.3914119526316],[5.266515731811523],"['KEARNEY_7_N001', 'ELCAJNGT_7_N004', 'CLARMNT..."
361,361,[171.93636761315796],[7.110429525375366],"['KEARNEY_7_N001', 'MISSION_6_N040', 'ELCAJNGT..."
362,362,[161.46235587894736],[4.261977672576904],"['KEARNEY_7_N001', 'ELCAJNGT_7_N004', 'CLARMNT..."
363,363,[123.12356490510876],[4.49529242515564],"['KEARNEY_7_N001', 'ELCAJNGT_7_N004', 'CLARMNT..."


In [17]:
lenth_list = [len(i.split(',')) for i in df_result4['out_set']]

In [19]:
sum(lenth_list)/len(lenth_list)

8.561643835616438